**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [52]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [53]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [54]:
# Get a list of available tables 
tables = list(client.list_tables(dataset)) # Your code here
list_of_tables = [table.table_id for table in tables] 

# Keep in mind versus the previous exercise that here the [] notation 
# accomplishes the same thing as the loop did previously 
#for table in tables:
    #print(table.table_id)

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [ ]:
# q_1.solution()

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [55]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,body,comment_count,community_owned_date,creation_date,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags
0,47949879,<p>Building onto what you have nearly got..</p...,0,None,2017-12-23 04:57:17.177000+00:00,2017-12-23 04:57:17.177000+00:00,NaT,,NaN,,5602957,47949810,2,2,
1,15503802,"<p>Normally, ""naturally aligned"" means that an...",1,None,2013-03-19 15:34:54.153000+00:00,2013-03-19 15:34:54.153000+00:00,NaT,,NaN,,179910,15503537,2,3,
2,3497784,"<p>Looking with Reflector, it seems to indicat...",0,None,2010-08-16 22:04:25.970000+00:00,2010-08-16 22:04:25.970000+00:00,NaT,,NaN,,1831,3497732,2,2,
3,39122150,<p>Probably not. I don't know what version of ...,1,None,2016-08-24 11:30:55.667000+00:00,2016-08-24 11:30:55.667000+00:00,2017-05-23 12:09:42.380000+00:00,,-1.0,,2868335,39110336,2,3,
4,36403800,"<p><code>replace(',', '')</code> only replaces...",0,None,2016-04-04 13:18:15.710000+00:00,2016-04-04 13:18:15.710000+00:00,NaT,,NaN,,2825245,36403626,2,2,


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [56]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count
0,9804136,"How do I clear an array of a structure in C, a...",<p>Two questions:</p>\n\n<ol>\n<li>How to quic...,9804289.0,3,2,None,2012-03-21 11:54:26.897000+00:00,NaN,2012-03-21 14:33:12.307000+00:00,2012-03-21 14:33:12.307000+00:00,,882600.0,,662586,1,1,c|struct,6462
1,44896935,How to implement threading to run two bash she...,<p>I have to record a wav file and at the same...,44897223.0,2,6,None,2017-07-04 04:14:50.020000+00:00,1.0,2017-07-11 09:23:08.210000+00:00,2017-07-11 09:23:08.210000+00:00,,3178797.0,,3178797,1,1,python|multithreading|bash|shell|python-multit...,411
2,3059091,How to remove carriage returns from output of ...,<p>I am using wordpress as a CMS and trying to...,3592816.0,7,5,None,2010-06-17 05:26:30.147000+00:00,11.0,2017-05-11 18:05:49.817000+00:00,2010-08-28 06:13:26.817000+00:00,,82330.0,,82330,1,26,php|javascript|wordpress|google-maps,73286
3,8926063,"Code Coverage: Why is end marker red (End If, ...",<p>I use MS-Test with Visual Studio 2010 and V...,8934375.0,4,0,None,2012-01-19 12:32:19.227000+00:00,NaN,2012-01-19 22:33:52.037000+00:00,2012-01-19 13:16:56.100000+00:00,,254041.0,,254041,1,4,vb.net|visual-studio|mstest|code-coverage,566
4,52977342,incorrect checksum for freed object - object w...,<p>I was hoping the return values in c++11 are...,NaN,0,10,None,2018-10-24 20:17:45.643000+00:00,1.0,2018-10-24 20:17:45.643000+00:00,NaT,,NaN,,1935611,1,0,c++|c++11,203


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [57]:
# tags
# Would need to use a string function in a conditional statement to subset
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [58]:
# Your code here
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id      ...      owner_user_id
0  39459578      ...          2438536.0
1  55002452      ...          7266985.0
2  36403258      ...          6156032.0
3  21106892      ...          3086806.0
4  30763305      ...          4929613.0

[5 rows x 3 columns]


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [ ]:
#q_3.hint()
#q_3.solution()

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [66]:
# Your code here
answers_query = """
        SELECT a.id, a.body, a.owner_user_id
        FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
        INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a 
            ON q.id = a.parent_id
        WHERE q.tags LIKE '%bigquery%'
"""

# The key is to recognize that the WHERE goes AFTER the join. It isnot like Groupby and Having. We can subset on the merge
# with WHERE

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
answers_query_job = client.query(answers_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe()  # Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

         id      ...      owner_user_id
0  44658435      ...          8060613.0
1  41475535      ...          4392693.0
2  45300212      ...            26003.0
3  50037327      ...          5265058.0
4  37287452      ...          4960314.0

[5 rows x 3 columns]


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [67]:
# q_4.hint()
# q_4.solution()

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [72]:
# Your code here
bigquery_experts_query = """
        SELECT a.owner_user_id AS user_id, COUNT(*) AS number_of_answers
        FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
        INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a 
            ON q.id = a.parent_id
        WHERE q.tags LIKE '%bigquery%'
        GROUP BY user_id
        HAVING number_of_answers > 0
        ORDER BY number_of_answers DESC
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

     user_id  number_of_answers
0  5221944.0               2338
1  6253347.0                698
2   132438.0                687
3  1366527.0                617
4   243782.0                450


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [71]:
#q_5.hint()
#q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

bigquery_experts_query = """
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id
                         """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

```

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [73]:
# You'd create an input function within the '% %' part for any term, then order it with DESC, and then hire

q_6.solution()

# You can create SQL functions!!!
    # Can input the topic with  + tag + in the query string
    # Can input the client with same setup as usual essentially

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%' + tag + '%'
               GROUP BY a.owner_user_id
               """
               
    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=1e9)      
    my_query_job = client.query(my_query, job_config=safe_config)
    
    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results

```

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=sizeAll&filetype=fileTypeBigQuery).

# Feedback

Bring any questions or feedback to the [Learn Discussion Forum](https://www.kaggle.com/learn-forum).

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*